Imports

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score
import random

In [ ]:
training_file_path = '/content/provided_train_clean.csv'
#training_file_path = '/content/provided_train.csv'
#training_file_path = '/content/obtained_train.csv'
#training_file_path = '/content/obtained_train_clean.csv'

testing_file_path = '/content/test_w_text.csv'

Preprocessing Training File

In [ ]:
df = pd.read_csv(training_file_path)
df.set_index('id', inplace = True)

In [ ]:
df.head()

,label,text
id,,
1139859146143076352,neutral,well done hk government for heeding concerns o...
1230398391751974912,positive,solidarity amp good luck with everyone startin...
1183039711193026560,neutral,having private health insurance doesnt mean a ...
1222618809162444800,negative,while president trump signed a check to cover ...
1163887022504906752,neutral,trump on omartlaib any jewish people that vote...


In [ ]:
df.label.value_counts()

neutral     10659
negative     7181
positive     5237
Name: label, dtype: int64

In [ ]:
possible_labels = df.label.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

{'neutral': 0, 'positive': 1, 'negative': 2}

In [ ]:
df.label = df['label'].map(label_dict)

In [ ]:
df.head()

,label,text
id,,
1139859146143076352,0,well done hk government for heeding concerns o...
1230398391751974912,1,solidarity amp good luck with everyone startin...
1183039711193026560,0,having private health insurance doesnt mean a ...
1222618809162444800,2,while president trump signed a check to cover ...
1163887022504906752,0,trump on omartlaib any jewish people that vote...


Preprocessing Testing File

In [ ]:
dftest = pd.read_csv(testing_file_path)
dftest.set_index('id', inplace = True)

In [ ]:
dftest.head()

,label,text
id,,
1222226229446750214,negative,Deputy White House counsel Pat Philbin on the ...
1236289649737371648,negative,Panic buying and stockpiling of toilet roll co...
1222258314857172993,negative,"It’s Super Bowl week, and Trump is using a pre..."
1236432648374857728,negative,Turns out Donald Trump was potentially exposed...
1233925225697488901,positive,Had a totally surreal author moment at B&amp;N...


In [ ]:
dftest.label.value_counts()

negative    496
positive    409
neutral     376
Name: label, dtype: int64

In [ ]:
possible_labels = dftest.label.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

{'negative': 0, 'positive': 1, 'neutral': 2}

In [ ]:
dftest.label = dftest['label'].map(label_dict)

In [ ]:
dftest.head()

,label,text
id,,
1222226229446750214,0,Deputy White House counsel Pat Philbin on the ...
1236289649737371648,0,Panic buying and stockpiling of toilet roll co...
1222258314857172993,0,"It’s Super Bowl week, and Trump is using a pre..."
1236432648374857728,0,Turns out Donald Trump was potentially exposed...
1233925225697488901,1,Had a totally surreal author moment at B&amp;N...


Train/Val Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.head()

,label,text,data_type
id,,,
1139859146143076352,0,well done hk government for heeding concerns o...,not_set
1230398391751974912,1,solidarity amp good luck with everyone startin...,not_set
1183039711193026560,0,having private health insurance doesnt mean a ...,not_set
1222618809162444800,2,while president trump signed a check to cover ...,not_set
1163887022504906752,0,trump on omartlaib any jewish people that vote...,not_set


In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

text
label data_type      
0     train      9060
      val        1599
1     train      4451
      val         786
2     train      6104
      val        1077

Load Tokenizer and Encode the Data

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    dftest.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)


In [ ]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(dftest.label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                            labels_val)

dataset_test = TensorDataset(input_ids_test,
                             attention_masks_test,
                             labels_test)

In [ ]:
dataset_val.tensors

(tensor([[ 101, 2383, 2797,  ...,    0,    0,    0],
         [ 101, 2096, 2343,  ...,    0,    0,    0],
         [ 101, 4067, 2017,  ...,    0,    0,    0],
         ...,
         [ 101, 2253, 2000,  ...,    0,    0,    0],
         [ 101, 2559, 2067,  ...,    0,    0,    0],
         [ 101, 6203, 5882,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 2, 0,  ..., 0, 1, 2]))

BERT Pretrained

In [ ]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased',
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Creating the DataLoaders

In [ ]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

dataloader_test = DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=32
)

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

Defining Our Performance Metrics

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

    print(f'Total Accuracy:{len(preds_flat[preds_flat==labels_flat])/len(labels_flat)}\n')
    print(metrics.confusion_matrix(labels_flat, preds_flat))
    print(metrics.classification_report(labels_flat, preds_flat, digits=3))



Create the Training Loop

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/4904 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0469313783002911


  0%|          | 0/109 [00:00<?, ?it/s]

Validation loss: 1.037527904597991
F1 Score (weighted): 0.3795262440750018


...

Epoch 5:   0%|          | 0/4904 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.542131662667789


  0%|          | 0/109 [00:00<?, ?it/s]

Validation loss: 2.8561758491971077
F1 Score (weighted): 0.3889460291269909


Epoch 6:   0%|          | 0/4904 [00:00<?, ?it/s]

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: negative
Accuracy:757/1599

Class: positive
Accuracy:221/786

Class: neutral
Accuracy:397/1077

Total Accuracy:0.39716926632004623

[[757 337 505]
 [322 221 243]
 [481 199 397]]
              precision    recall  f1-score   support

           0      0.485     0.473     0.479      1599
           1      0.292     0.281     0.286       786
           2      0.347     0.369     0.357      1077

    accuracy                          0.397      3462
   macro avg      0.375     0.374     0.374      3462
weighted avg      0.398     0.397     0.398      3462



Testing results

In [ ]:
test_loss, test_predictions, test_true_vals = evaluate(dataloader_test)
test_f1 = f1_score_func(test_predictions, test_true_vals)
print(f'Validation loss: {test_loss}')
print(f'F1 Score (weighted): {test_f1}')

  0%|          | 0/41 [00:00<?, ?it/s]

Validation loss: 5.665942866627763
F1 Score (weighted): 0.37952904761330714


In [ ]:
accuracy_per_class(test_predictions, test_true_vals)

Class: negative
Accuracy:182/496

Class: positive
Accuracy:151/409

Class: neutral
Accuracy:150/376

Total Accuracy:0.3770491803278688

[[182  78 236]
 [154 151 104]
 [129  97 150]]
              precision    recall  f1-score   support

           0      0.391     0.367     0.379       496
           1      0.463     0.369     0.411       409
           2      0.306     0.399     0.346       376

    accuracy                          0.377      1281
   macro avg      0.387     0.378     0.379      1281
weighted avg      0.389     0.377     0.380      1281

